# Basic LLM Experiment

Here, we run a experiment with a llm model via api.

## Define the experiment

First, we define the experiment.

In [ ]:
from sweetbean.sequence import Block, Experiment
from sweetbean.stimulus import TextStimulus
from sweetbean.parameter import TimelineVariable, DataVariable, DerivedLevel, DerivedParameter

# TIMELINE
timeline = [
    {"color": "red", "word": "RED", "correct_key": "f"},
    {"color": "green", "word": "GREEN", "correct_key": "j"},
    {"color": "green", "word": "RED", "correct_key": "f"},
    {"color": "red", "word": "GREEN", "correct_key": "j"},
]

# EVENT SEQUENCE

color = TimelineVariable("color", ["red", "green"])
word = TimelineVariable("word", ["RED", "GREEN"])


def is_correct_f(color):
    return color == "red"


def is_correct_j(color):
    return not is_correct_f(color)


j_key = DerivedLevel("j", is_correct_j, [color])
f_key = DerivedLevel("f", is_correct_f, [color])

correct_key = DerivedParameter("correct", [j_key, f_key])

# Creating a data variable
correct = DataVariable("correct", [True, False])


# Predicates
def is_correct(correct):
    return correct


def is_false(correct):
    return not correct


# Derived Levels
correct_feedback = DerivedLevel("correct", is_correct, [correct], 2)
false_feedback = DerivedLevel("false", is_false, [correct], 2)

# Derived Parameter
feedback_text = DerivedParameter("feedback_text", [correct_feedback, false_feedback])

# Using it in the stimulus
fixation = TextStimulus(1000, "+")

so_s = TextStimulus(400)
stroop = TextStimulus(2000, word, color, ["j", "f"], correct_key)
so_f = TextStimulus(300)
feedback = TextStimulus(800, feedback_text)

event_sequence = [fixation, so_s, stroop, so_f, feedback]

# BLOCK DESIGN

train_block = Block(event_sequence, timeline)
experiment = Experiment([train_block])

To test what the LLM "sees", we can run the experiment as chat on ourselves. If we set multiturn to true we will not see the full chat history but only the last generated prompt.

In [2]:
data = experiment.run_on_language(get_input=input, multiturn=True)
data

{'full_chat': ' You see "+" written in white. You see a blank screen. You see "RED" written in red. You can press [\'j\', \'f\']. You press <<>>. You see a blank screen. You see "false" written in white. You see "+" written in white. You see a blank screen. You see "GREEN" written in green. You can press [\'j\', \'f\']. You press <<>>. You see a blank screen. You see "false" written in white. You see "+" written in white. You see a blank screen. You see "RED" written in green. You can press [\'j\', \'f\']. You press <<>>. You see a blank screen. You see "false" written in white. You see "+" written in white. You see a blank screen. You see "GREEN" written in red. You can press [\'j\', \'f\']. You press <<>>. You see a blank screen. You see "false" written in white.',
 'data_lst': {'choices': [[],
   [],
   ['j', 'f'],
   [],
   [],
   [],
   [],
   ['j', 'f'],
   [],
   [],
   [],
   [],
   ['j', 'f'],
   [],
   [],
   [],
   [],
   ['j', 'f'],
   [],
   []],
  'type': ['jsPsychHtmlKey

Let's run the experiment on the AI. Here, we use api calls to google cloud, but all that is needed is a function that generates text from other text. In this case `generate` uses the input and makes an api call and returns the response.

In [3]:
from google_api import generate


def parse_response(response, correct_key):
 return correct_key in response


data_ai = experiment.run_on_language(get_input=generate, parse_response=parse_response,
                                     intro="You are a participant in a psychological experiment. You're goal is not to react as accurate as possible but as similar to a human as possible. This includes making similar mistakes as humans. You will be given a sequence of things you see and your goal is to react with a single letter indicating a key press. Your response should always be a single letter. Please respond with the keys 'j' and 'f' to indicate the color of the word. The correct key for red is 'f' and for green 'j'. Please indicate your response by either f or j.")
data_ai

/Users/younesstrittmatter/Documents/GitHub/AutoResearch/sweetbean/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


{'full_chat': 'You are a participant in a psychological experiment. You\'re goal is not to react as accurate as possible but as similar to a human as possible. This includes making similar mistakes as humans. You will be given a sequence of things you see and your goal is to react with a single letter indicating a key press. Your response should always be a single letter. Please respond with the keys \'j\' and \'f\' to indicate the color of the word. The correct key for red is \'f\' and for green \'j\'. Please indicate your response by either f or j. You see "+" written in white. You see a blank screen. You see "RED" written in red. You can press [\'j\', \'f\']. You press <<f>>. You see a blank screen. You see "correct" written in white. You see "+" written in white. You see a blank screen. You see "GREEN" written in green. You can press [\'j\', \'f\']. You press <<j>>. You see a blank screen. You see "correct" written in white. You see "+" written in white. You see a blank screen. You